In [101]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `ssh://git@github.com/JuliaRegistries/General.git`


In [102]:
using LinearAlgebra
using QPControl
using QPControl.Trajectories
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using MathOptInterface
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
using OSQP
using PushRecovery
using PlanarConvexHulls
using Rotations
BLAS.set_num_threads(1)

In [103]:
if !(@isdefined mechanism)
    mechanism = AtlasRobot.mechanism(add_flat_ground=true);
end

# static contact information
soleframes = add_sole_frames!(mechanism)
foot_polygons = make_foot_polygons(mechanism, soleframes; num_extreme_points=4);

# body lookup
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis");

In [104]:
# create low level controller
lowlevel = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    lowlevel = MomentumBasedController{4}(mechanism, optimizer,
        floatingjoint = findjoint(mechanism, "pelvis_to_world"));
    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
            μ = point.model.friction.μ
            contact = addcontact!(lowlevel, body, position, normal, μ)
            contact.maxnormalforce[] = 1e6 # TODO
            contact.weight[] = 1e-3
        end
    end
    lowlevel
end;

In [105]:
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)

MechanismState{Float64, Float64, Float64, …}(…)

In [106]:
# ICP stuff
zdes = center_of_mass(nominalstate).v[3] - 0.05
gz = norm(mechanism.gravitational_acceleration)
ω = sqrt(gz / zdes)
icptraj = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-6)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-8)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 10000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    max_polygon_sides = 6 # TODO
    num_segments = 9
    ICPTrajectoryGenerator{Float64, max_polygon_sides}(optimizer, num_segments, ω)
end
linear_momentum_controller = ICPController(mechanism, icptraj, zdes);

In [107]:
# walking state machine
statemachine = let
    contacts = Dict(BodyID(body) => contact for (body, contact) in lowlevel.contacts)
    ICPWalkingStateMachine(mechanism, contacts, icptraj)
end;

In [108]:
PushRecovery.init_footstep_plan!(statemachine, nominalstate, foot_polygons);

In [109]:
# create high level controller
controller = PushRecoveryController(lowlevel, pelvis, nominalstate, 
    statemachine, collect(values(statemachine.end_effector_controllers)), linear_momentum_controller);

In [110]:
# create visualizer
using MeshCat
using MeshCatMechanisms
using MechanismGeometries

if !(@isdefined gui) || !any(isopen, MeshCatMechanisms.visualizer(gui.visualizer).core.scope.pool.connections)
    pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0))
    link_colors = Dict(map(foot -> string(foot) => RGBA(0.7f0, 0.7f0, 0.7f0, 0.3f0), feet))
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path=[AtlasRobot.packagepath()], link_colors=link_colors)
    vis = Visualizer()
    mvis = MechanismVisualizer(mechanism, visuals, vis)
    pvis = PushRecoveryVisualizer(mvis)
    gui = GUI(pvis, usernode=Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    open(gui)
end
copyto!(gui.visualizer, nominalstate);

In [111]:
# create ODEProblem
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)
Δt = 1 / 300
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
# dynamics = Dynamics(mechanism, pcontroller)
dynamics = Dynamics(mechanism, SumController(similar(velocity(state)), (pcontroller, pushapplier)))
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui))
problem = ODEProblem(dynamics, state, (0., 15.0); callback=callback)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 15.0)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [112]:
# simulate
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
last(sol.t)

 15.534481 seconds (16.76 M allocations: 1.022 GiB, 8.00% gc time)


15.0

In [113]:
setanimation!(mvis, sol)

In [114]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult